# 2---ไฟล์ Model_Training เป็นไฟล์ที่ใช้ Train Model ที่ Create ไว้ 

In [ ]:
import numpy as np
import IPython
from n_utils import *
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam
import matplotlib.pyplot as plt
%matplotlib inline

## ทำการ Build model

In [ ]:
Tx = 5511
Ty = 1375
n_freq= 101

In [ ]:
def model(input_shape):

    X_input = Input(shape = input_shape)
    X = Conv1D(196, kernel_size=15, strides=4)(X_input)                               
    X = BatchNormalization()(X)                              
    X = Activation('relu')(X)                                
    X = Dropout(0.5)(X)                                 

    X = GRU(units = 128, return_sequences = True, reset_after=True)(X) 
    X = Dropout(0.5)(X)                                 
    X = BatchNormalization()(X)                                

    X = GRU(units = 128, return_sequences = True, reset_after=True)(X)  
    X = Dropout(0.5)(X)                                
    X = BatchNormalization()(X)                              
    X = Dropout(0.5)(X)                               

    X = TimeDistributed(Dense(1, activation = "sigmoid"))(X)

    model = Model(inputs = X_input, outputs = X)
    
    return model  

In [ ]:
model = model(input_shape = (Tx, n_freq))

In [ ]:
model.summary()

## โหลดไฟล์ที่ Train และ Test 

In [ ]:
X = np.load('D:/Babie/DetectHelp/Code_No/Train/XY_train/X_modelh.npy')
Y = np.load('D:/Babie/DetectHelp/Code_No/Train/XY_train/Y_modelh.npy')
Y.shape

In [ ]:
X_dev = np.load('D:/Babie/DetectHelp/Code_No/Train/XY_dev/X_model_devh.npy')
Y_dev = np.load('D:/Babie/DetectHelp/Code_No/Train/XY_dev/Y_model_devh.npy')
Y_dev.shape

## โหลด Model เก่ามา Train กับข้อมูลใหม่

In [ ]:
model=load_model('D:/Babie/DetectHelp/Code_No/models/No_11.h5')

## Fit Model

#### learning_rate
- ถ้า Learning Rate มีค่าน้อย Weight ของโมเดลก็จะเปลี่ยนแปลงน้อย การทำงานของโมเดลก็จะเปลี่ยนไปน้อย Loss ก็ไม่ค่อยเปลี่ยนเท่าไร
- ถ้า Learning Rate มีค่ามาก Weight ของโมเดลก็จะเปลี่ยนแปลงมาก การทำงานของโมเดลก็จะเปลี่ยนไปมาก Loss ก็จะเปลี่ยนแปลงมาก

#### batch_size
- เวลาที่ใช้เทรน – Batch Size เล็กช้ากว่า
- Memory ที่ใช้ – Batch Size เล็กใช้น้อยกว่า
- และที่สำคัญ ความแม่นยำของโมเดล – Batch Size ขนาด <=32 แม่นยำกว่า

In [ ]:
#fit the model
opt = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])
model.fit(X, Y, batch_size = 32, epochs=15)

## ได้ Model มาแล้ว Save Model

In [ ]:
model.save('D:/Babie/DetectHelp/Code_No/models/No_12.h5')

## Test Model

In [ ]:
#test the model
loss, acc = model.evaluate(X_dev, Y_dev)
print("Dev set accuracy = ", acc)